## Thuật toán newton tiến cho phương pháp lặp để giải đa thức nội suy 

In [11]:
import numpy as np
import pandas as pd
import math
from numpy.polynomial import polynomial as P

def load_universal_data(filepath):
    """
    Hàm đọc dữ liệu mạnh mẽ, tự động phát hiện định dạng hàng ngang hoặc dọc.
    - Xử lý dấu phẩy (,) và dấu chấm (.).
    - Xử lý ký tự BOM (Byte Order Mark) từ Excel.
    - Bỏ qua các dòng trống hoặc dòng tiêu đề.
    """
    processed_lines = []
    try:
        with open(filepath, 'r', encoding='utf-8-sig') as f:
            all_lines = f.readlines()
            for line in all_lines:
                line = line.strip()
                if not line: continue
                line = line.replace(',', '.')
                processed_lines.append(line)
                
    except FileNotFoundError:
        raise IOError(f"LỖI: Không tìm thấy file '{filepath}'.")
    except Exception as e:
        raise IOError(f"LỖI: Không thể đọc file '{filepath}'. Lỗi: {e}")

    if len(processed_lines) == 0: return np.array([])
    elif len(processed_lines) == 1:
        # --- ĐỊNH DẠNG HÀNG NGANG ---
        print(f"    (Phát hiện định dạng hàng ngang trong file {filepath})")
        try:
            return np.fromstring(processed_lines[0], sep=' ')
        except Exception as e:
            raise ValueError(f"LỖI: Dữ liệu hàng ngang trong file '{filepath}' không hợp lệ.")
    else:
        # --- ĐỊNH DẠNG HÀNG DỌC ---
        print(f"    (Phát hiện định dạng hàng dọc trong file {filepath})")
        data = []
        for i, line in enumerate(processed_lines):
            try:
                value = float(line)
                data.append(value)
            except ValueError:
                print(f"    (Cảnh báo: Bỏ qua dòng {i+1} không hợp lệ: '{line}')")
                pass
        return np.array(data)

# --- Chương trình chính cho Ô nạp dữ liệu ---
try:
    # 1. Nạp dữ liệu thô
    x_coords_raw = load_universal_data('FileX.txt')
    y_coords_raw = load_universal_data('FileY.txt')

    if len(x_coords_raw) == 0 or len(y_coords_raw) == 0:
        raise ValueError("LỖI: Một trong hai file không chứa dữ liệu số hợp lệ.")
    if len(x_coords_raw) != len(y_coords_raw):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")
    
    # 2. SẮP XẾP TĂNG DẦN (cho Newton Tiến và tìm h)
    print("Đang sắp xếp dữ liệu theo x tăng dần...")
    sort_indices = np.argsort(x_coords_raw)
    x_coords = x_coords_raw[sort_indices]
    y_coords = y_coords_raw[sort_indices]
    
    # 3. KIỂM TRA MỐC CÁCH ĐỀU (BẮT BUỘC)
    diffs = np.diff(x_coords)
    if not np.allclose(diffs, diffs[0], rtol=1e-08, atol=1e-10):
        raise ValueError(f"LỖI: Các mốc nội suy x không cách đều nhau. Phương pháp lặp không thể áp dụng.")
    
    h_step = diffs[0] # Lưu lại bước nhảy h (sẽ là số dương)

    # 4. In kết quả đã sắp xếp
    print("Dữ liệu đã được đọc, sắp xếp và kiểm tra cách đều THÀNH CÔNG!")
    print(f"Tổng số {len(x_coords)} điểm đã được nạp. Bước nhảy h = {h_step:g}")
    print("-" * 50)
    print("Các mốc x (đã sắp xếp tăng dần):", x_coords.tolist())
    print("Các giá trị y (tương ứng):", y_coords.tolist())
    
except (IOError, ValueError) as e:
    print(str(e))

    (Phát hiện định dạng hàng dọc trong file FileX.txt)
    (Phát hiện định dạng hàng dọc trong file FileY.txt)
Đang sắp xếp dữ liệu theo x tăng dần...
Dữ liệu đã được đọc, sắp xếp và kiểm tra cách đều THÀNH CÔNG!
Tổng số 7 điểm đã được nạp. Bước nhảy h = 0.118
--------------------------------------------------
Các mốc x (đã sắp xếp tăng dần): [3.596, 3.714, 3.832, 3.95, 4.068, 4.186, 4.304]
Các giá trị y (tương ứng): [3.1948, 2.8267, 2.4384, 2.0354, 1.6234, 1.2084, 0.7962]


In [12]:
def create_full_difference_table(y_values):
    """Tạo bảng sai phân tiến đầy đủ (dạng ma trận trên)."""
    num_points = len(y_values)
    table = [[0.0 for _ in range(num_points)] for _ in range(num_points)]
    
    # Cột đầu tiên là y
    for i in range(num_points):
        table[i][0] = y_values[i]
        
    # Tính các cột sai phân
    for j in range(1, num_points): # Cột (bậc)
        for i in range(num_points - j): # Hàng
            # Công thức sai phân tiến: Δ^j y_i = Δ^{j-1} y_{i+1} - Δ^{j-1} y_i
            table[i][j] = table[i+1][j-1] - table[i][j-1]
            
    return np.array(table)

# --- Chương trình chính cho Ô 2 ---
try:
    # Tính bảng sai phân đầy đủ
    diff_table_for_calc = create_full_difference_table(y_coords)
    
    print("--- Bảng sai phân đầy đủ (dạng tam giác trên) ---")
    display(pd.DataFrame(diff_table_for_calc).style.format("{:g}", na_rep=""))
    print("Bảng sai phân đã được tính và lưu vào 'diff_table_for_calc'.")

except (NameError) as e:
    print(f"LỖI: {e}. Vui lòng chạy lại Ô 1 trước.")

--- Bảng sai phân đầy đủ (dạng tam giác trên) ---


,0,1,2,3,4,5,6
0,3.1948,-0.3681,-0.0202,0.0055,0.0002,0.0001,-0.0006
1,2.8267,-0.3883,-0.0147,0.0057,0.0003,-0.0005,0
2,2.4384,-0.403,-0.009,0.006,-0.0002,0,0
3,2.0354,-0.412,-0.003,0.0058,0,0,0
4,1.6234,-0.415,0.0028,0,0,0,0
5,1.2084,-0.4122,0,0,0,0,0
6,0.7962,0,0,0,0,0,0


Bảng sai phân đã được tính và lưu vào 'diff_table_for_calc'.


In [13]:
def phi_function(t, y_bar, coeffs_row_k):
    """
    Tính giá trị của hàm lặp phi(t) dựa trên công thức Newton tiến.
    coeffs_row_k là mảng [y_k, Δy_k, Δ^2 y_k, ...]
    """
    y_k = coeffs_row_k[0]
    delta_y_k = coeffs_row_k[1]
    
    if np.isclose(delta_y_k, 0):
        # Trường hợp hiếm gặp, nếu Δy_k = 0 thì không thể chia
        return np.nan 
    
    # Tính phần trong ngoặc [...]
    # (Δ^2*y_k/2!)*t*(t-1) + (Δ^3*y_k/3!)*t*(t-1)*(t-2) + ...
    bracket_sum = 0.0
    
    # Lặp từ i=2 (bậc sai phân 2)
    for i in range(2, len(coeffs_row_k)):
        # Tính tích t(t-1)...(t-i+1)
        t_product = 1.0
        for j in range(i):
            t_product *= (t - j)
        
        # Thêm số hạng (Δ^i y_k / i!) * t_product
        if i < len(coeffs_row_k): # Đảm bảo không vượt quá mảng
            bracket_sum += (coeffs_row_k[i] / math.factorial(i)) * t_product
        
    # Áp dụng công thức lặp đầy đủ 
    t_new = (y_bar - y_k) / delta_y_k - (1 / delta_y_k) * bracket_sum
    
    return t_new

print("Hàm lặp phi(t) đã được định nghĩa.")

Hàm lặp phi(t) đã được định nghĩa.


In [14]:
def print_phi_formula(y_bar, coeffs_row_k):
    """
    [cite_start]In ra công thức tượng trưng của hàm lặp phi(t)[cite: 538, 554].
    coeffs_row_k là mảng [y_k, Δy_k, Δ^2 y_k, ...]
    """
    print("\n" + "="*50)
    print("--- CÔNG THỨC LẶP φ(t) TƯƠNG ỨNG ---")
    
    y_k = coeffs_row_k[0]
    delta_y_k = coeffs_row_k[1]
    
    if np.isclose(delta_y_k, 0):
        print("LỖI: Δy_k bằng 0, không thể xây dựng công thức lặp.")
        return

    # 1. Tính toán các hằng số
    term_t0 = (y_bar - y_k) / delta_y_k
    term_multiplier = -1.0 / delta_y_k
    
    # 2. Bắt đầu xây dựng chuỗi
    formula_str = f"φ(t) = {term_t0:g}"
    
    # 3. Xây dựng phần trong ngoặc [...]
    bracket_str = ""
    for i in range(2, len(coeffs_row_k)):
        diff_val = coeffs_row_k[i]
        
        # Bỏ qua nếu sai phân này bằng 0
        if np.isclose(diff_val, 0):
            continue
            
        # Tính hệ số của số hạng (Δ^i y_k / i!)
        term_coeff = diff_val / math.factorial(i)
        
        # Xử lý dấu
        sign = " - " if term_coeff < 0 else " + "
        if bracket_str == "": # Xử lý dấu cho số hạng đầu tiên
            sign = "-" if term_coeff < 0 else ""
            
        # Xây dựng tích t(t-1)...(t-i+1)
        t_product_terms = []
        for j in range(i):
            if j == 0:
                t_product_terms.append("t")
            else:
                t_product_terms.append(f"(t-{j})")
        t_product_str = "*".join(t_product_terms)
        
        # Nối vào chuỗi trong ngoặc
        bracket_str += f"{sign} {abs(term_coeff):g} * {t_product_str} "
        
    # 4. Hoàn thành công thức
    if bracket_str: # Chỉ thêm phần ngoặc nếu nó không rỗng
        sign_multiplier = " - " if term_multiplier < 0 else " + "
        formula_str += f"{sign_multiplier} {abs(term_multiplier):g} * [ {bracket_str}]"
        
    print(formula_str)
    print("="*50)

print("Hàm in công thức lặp 'print_phi_formula' đã được định nghĩa.")

Hàm in công thức lặp 'print_phi_formula' đã được định nghĩa.


In [16]:
# Giả định các biến x_coords, y_coords, h_step, diff_table_for_calc, phi_function
# và print_phi_formula đã được định nghĩa ở các ô trước.

def find_isolation_interval(y_arr, y_val, tol=1e-6):
    """
    Tìm chỉ số k sao cho:
      - y_arr[k] <= y_val <= y_arr[k+1] nếu y tăng dần
      - y_arr[k] >= y_val >= y_arr[k+1] nếu y giảm dần
    Trả về -1 nếu ngoài khoảng hoặc y không đơn điệu.
    """
    n = len(y_arr)
    if n < 2:
        return -1

    # Kiểm tra đơn điệu
    dy = np.diff(y_arr)
    is_increasing = np.all(dy >= -tol)
    is_decreasing = np.all(dy <= tol)

    if not (is_increasing or is_decreasing):
        # In để debug nếu cần
        print("LỖI: Dữ liệu y không đơn điệu. Không thể tìm khoảng cách ly tự động.")
        return -1

    if is_increasing:
        # kiểm tra ngoài khoảng cho chuỗi tăng
        if y_val < y_arr[0] - tol or y_val > y_arr[-1] + tol:
            return -1
        for k in range(n - 1):
            if (y_arr[k] - tol) <= y_val <= (y_arr[k + 1] + tol):
                return k
    else:
        # chuỗi giảm
        if y_val < y_arr[-1] - tol or y_val > y_arr[0] + tol:
            return -1
        for k in range(n - 1):
            if (y_arr[k + 1] - tol) <= y_val <= (y_arr[k] + tol):
                return k

    return -1


# --- Chương trình chính cho Ô 4 ---
try:
    # --- Thiết lập các tham số cho vòng lặp ---
    max_iterations = 1000000
    tolerance = 1e-5
    
    # --- 1. Nhập giá trị y_bar ---
    y_prime_input = float(input(f"Nhập giá trị y (y_bar) bạn muốn tìm x: "))
    
    # --- 2. Tìm khoảng cách ly ---
    # Nếu k_index chưa tồn tại, tự tìm
    
    k_index = find_isolation_interval(y_coords, y_prime_input)
    
    try:
        k_index
    except NameError:
        k_index = find_isolation_interval(y_coords, y_prime_input)

    if k_index == -1:
        raise ValueError(f"LỖI: Không tìm thấy khoảng cách ly nào cho y = {y_prime_input}.")
        
    x_k = x_coords[k_index]
    y_k = y_coords[k_index]
    forward_coeffs_k = diff_table_for_calc[k_index, :]

    print(f"Giá trị y_bar = {y_prime_input} nằm trong khoảng (k={k_index}).")
    print(f"==> Sử dụng mốc bắt đầu là x_k = {x_k}, y_k = {y_k}.")
    print(f"Sử dụng vector hệ số (hàng {k_index}): {forward_coeffs_k}")

    # Gọi hàm in công thức (từ Ô 3.5)
    print_phi_formula(y_prime_input, forward_coeffs_k)

    # --- 3. Thực hiện vòng lặp ---
    delta_y_k = forward_coeffs_k[1]
    if np.isclose(delta_y_k, 0):
        raise ValueError("LỖI: Δy_k bằng 0. Phương pháp lặp thất bại.")
        
    t_old = (y_prime_input - y_k) / delta_y_k
    print(f"\n--- Bắt đầu lặp với t_0 = {t_old:g} ---")
    
    # >>> THAY ĐỔI: Tạo danh sách để lưu lịch sử lặp <<<
    iteration_log = []
    iteration_log.append([0, t_old, np.nan]) # Thêm t_0, sai số chưa có

    converged = False
    x_result = np.nan
    t_final = t_old

    for j in range(max_iterations):
        t_new = phi_function(t_old, y_prime_input, forward_coeffs_k)
        
        if np.isnan(t_new):
            print(f"Lỗi tính toán trong hàm phi(t). Dừng lặp.")
            break
            
        # Tính sai số
        error = abs(t_new - t_old)
        
        # >>> THAY ĐỔI: Ghi lại lịch sử lặp <<<
        iteration_log.append([j + 1, t_new, error])
        
        if error < tolerance:
            print(f"\nĐã hội tụ sau {j+1} lần lặp.")
            converged = True
            t_final = t_new
            x_result = x_k + t_final * h_step
            break # Thoát khỏi vòng lặp
            
        t_old = t_new
        
    # --- B4: In Bảng Lịch sử Lặp ---
    print("\n" + "="*50)
    print("--- BẢNG LỊCH SỬ LẶP ---")
    df_iterations = pd.DataFrame(iteration_log, columns=['k (lần lặp)', 't_k', '|t_k - t_{k-1}|'])
    
    # Chỉ định dạng cho các cột số
    numeric_cols = ['t_k', '|t_k - t_{k-1}|']
    display(df_iterations.style.format("{:g}", subset=numeric_cols, na_rep="---"))
    
    # --- B5: In Kết quả Cuối cùng ---
    if converged:
        print("\n" + "="*50)
        print("--- KẾT QUẢ NỘI SUY NGƯỢC ---")
        print(f"Giá trị t cuối cùng: t = {t_final:g}")
        print(f"Kết quả:   x = {x_k:g} + {t_final:g} * {h_step:g} = {x_result:g}")
        print(f"\n>>> Giá trị x cần tìm là: x ≈ {x_result:g} <<<")
        print("="*50)
    elif np.isnan(t_new):
        pass # Đã in lỗi tính toán ở trên
    else:
        print(f"\nLỖI: Phương pháp lặp không hội tụ sau {max_iterations} lần lặp.")

except (ValueError, NameError, IOError) as e:
    print(f"LỖI: {e}. Vui lòng chạy lại các ô trước đó.")

Giá trị y_bar = 3.0 nằm trong khoảng (k=0).
==> Sử dụng mốc bắt đầu là x_k = 3.596, y_k = 3.1948.
Sử dụng vector hệ số (hàng 0): [ 3.1948e+00 -3.6810e-01 -2.0200e-02  5.5000e-03  2.0000e-04  1.0000e-04
 -6.0000e-04]

--- CÔNG THỨC LẶP φ(t) TƯƠNG ỨNG ---
φ(t) = 0.529204 +  2.71665 * [ - 0.0101 * t*(t-1)  +  0.000916667 * t*(t-1)*(t-2)  +  8.33333e-06 * t*(t-1)*(t-2)*(t-3)  +  8.33333e-07 * t*(t-1)*(t-2)*(t-3)*(t-4)  -  8.33333e-07 * t*(t-1)*(t-2)*(t-3)*(t-4)*(t-5) ]

--- Bắt đầu lặp với t_0 = 0.529204 ---

Đã hội tụ sau 3 lần lặp.

--- BẢNG LỊCH SỬ LẶP ---


,k (lần lặp),t_k,|t_k - t_{k-1}|
0,0,0.529204,---
1,1,0.536971,0.00776712
2,2,0.53695,2.1128e-05
3,3,0.53695,6.25376e-08



--- KẾT QUẢ NỘI SUY NGƯỢC ---
Giá trị t cuối cùng: t = 0.53695
Kết quả:   x = 3.596 + 0.53695 * 0.118 = 3.65936

>>> Giá trị x cần tìm là: x ≈ 3.65936 <<<
